In [2]:
# https://www.youtube.com/watch?v=kCc8FmEb1nY&ab_channel=AndrejKarpathy

In [3]:
!ls

gpt-dev.ipynb  input.txt


In [4]:
# have downloaded the tiny shakespeare dataset

In [5]:
# read in the file to inspect it
with open("input.txt", mode="r", encoding="utf-8") as f:
    text = f.read()

In [6]:
len(text)

1115394

In [7]:
text[:100]

'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'

In [8]:
# create a list of the unique characters within the text
chars = sorted(list(set(text)))
# get the size of the vocabulary
vocab_size = len(chars)

In [9]:
print("".join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [10]:
# tokenise the input text
# convert the raw text string into a sequence of integers according to the vocabulary of possible elements
# character level language model. We are going to convert individual characters into integers

In [11]:
# create a mapping from characters to integers. This is the tokeniser
stoi = { ch:i for i, ch in enumerate(chars)} # strings to integers. This makes a dict with chars as the keys and the
# corresponding integer as the value.

itos = { i:ch for i, ch in enumerate(chars)} # ints to strings. Creates a dict with int keys and char values.

encode = lambda s: [stoi[c] for c in s] # converts a string of chars to ints, by character
decode = lambda s: "".join([itos[i] for i in s]) # converts a list of ints to chars, by character and joins them

In [16]:
print(encode("Hello there!"))
print(decode([20, 43, 50, 50, 53, 1, 58, 46, 43, 56, 43, 2]))

[20, 43, 50, 50, 53, 1, 58, 46, 43, 56, 43, 2]
Hello there!


In [21]:
encode(text)[:10]

[18, 47, 56, 57, 58, 1, 15, 47, 58, 47]

In [17]:
# We will encode the text file and wrap it in a pytorch tensor
# pytorch tensors are multi-dimensional arrays that can be operated on by cuda-capable GPUs.
# The GPUs have hundreds of cores, which can collectively run thousands of threads.
import torch
data = torch.tensor(encode(text), dtype=torch.long)

In [18]:
# separate the data into train and test split
n = int(0.9*len(data)) # take the first 90% of the data
train_data = data[:n] # training data
val_data = data[n:] # validation data

In [19]:
block_size = 8 # the size of data which will be passed to the transformer
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [20]:
x = train_data[:block_size] # x is the input
y = train_data[1:block_size+1] # y is the target for each position in the input
for t in range(block_size):
    context = x[:t+1] # context is all of the characters up to and including t
    target = y[t] # the target is the t'th character in the targets array, y
    print(f"when input is {context}, the target is: {target}")
# this trains the transformer on contexts from as little as 1, up to block_size

when input is tensor([18]), the target is: 47
when input is tensor([18, 47]), the target is: 56
when input is tensor([18, 47, 56]), the target is: 57
when input is tensor([18, 47, 56, 57]), the target is: 58
when input is tensor([18, 47, 56, 57, 58]), the target is: 1
when input is tensor([18, 47, 56, 57, 58,  1]), the target is: 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]), the target is: 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]), the target is: 58


In [22]:
# We will feed multiple chunks of text (blocks) to the transformer at a time, in batches.
# This is efficient as it uses more of the GPU as data can be processed in parallel.
# The chunks of data are processed independently and do not interact or talk to each other.

In [26]:
torch.manual_seed(1337)
batch_size = 4 # how many independent sequences we will process in parallel, blocks, in every forward backward pass
# in the transformer
block_size = 8 # what is the maximum context length for predictions

def get_batch(split):
    # generate a small batch of data of inputs, x, and targets, y
    data = train_data if split == 'train' else val_dat
    xi = torch.randint(len(data) - block_size, (batch_size, 1)) # creates 4 (batch_size) row starting positions 
    # from the text data
    x = torch.stack([data[i:i+block_size] for i in xi])
    y = torch.stack([data[i+1:i+block_size+1] for i in xi])
    return x, y

In [38]:
torch.manual_seed(1337)
xb, yb = get_batch('train')

for b in range(batch_size):
    for t in range(block_size):
        context = xb[b, :t+1]
        target = yb[b, t]
        print(f"when input is {context}, target is {target}")        

when input is tensor([24]), target is 43
when input is tensor([24, 43]), target is 58
when input is tensor([24, 43, 58]), target is 5
when input is tensor([24, 43, 58,  5]), target is 57
when input is tensor([24, 43, 58,  5, 57]), target is 1
when input is tensor([24, 43, 58,  5, 57,  1]), target is 46
when input is tensor([24, 43, 58,  5, 57,  1, 46]), target is 43
when input is tensor([24, 43, 58,  5, 57,  1, 46, 43]), target is 39
when input is tensor([44]), target is 53
when input is tensor([44, 53]), target is 56
when input is tensor([44, 53, 56]), target is 1
when input is tensor([44, 53, 56,  1]), target is 58
when input is tensor([44, 53, 56,  1, 58]), target is 46
when input is tensor([44, 53, 56,  1, 58, 46]), target is 39
when input is tensor([44, 53, 56,  1, 58, 46, 39]), target is 58
when input is tensor([44, 53, 56,  1, 58, 46, 39, 58]), target is 1
when input is tensor([52]), target is 58
when input is tensor([52, 58]), target is 1
when input is tensor([52, 58,  1]), tar

In [39]:
# our input to the transformer
xb

tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])

In [ ]:
# https://youtu.be/kCc8FmEb1nY?t=1330